# TP4, INF8225 2025, Projet


## Imports

In [1]:
from IPython.display import clear_output

%pip install numpy
%pip install swig
%pip install box2d
%pip install pygame
%pip install gymnasium
%pip install "gymnasium[box2d]"
%pip install matplotlib

clear_output()

In [2]:
import numpy as np
import gymnasium as gym
import matplotlib.pyplot as plt
from IPython.display import HTML
import matplotlib.animation as animation

## Data Declaration

In [5]:
# Inspired by : https://github.com/pangyyen/carRacing-DeepRL/blob/main/ppo/ppo.ipynb

env = gym.make("CarRacing-v3", render_mode="rgb_array", domain_randomize=False, continuous=True)
print("Observation space: ", env.observation_space) # (low, high, shape, dtype)
print("Action space: ", env.action_space)

SEED = 42

go_up = np.array([0.0, 1.0, 0.0])
no_action = np.array([0.0, 0.0, 0.0])

print("\nInitial State")
observation, info = env.reset(seed=SEED)

def show_animation(frames):
	fig = plt.figure(figsize=(5, 5))
	plt.axis('off')
	im = plt.imshow(frames[0])

	def animate(i):
		im.set_data(frames[i])
		return im,

	anim = animation.FuncAnimation(fig, animate, frames=len(frames))
	plt.close(fig)
	display(HTML(anim.to_jshtml()))

Observation space:  Box(0, 255, (96, 96, 3), uint8)
Action space:  Box([-1.  0.  0.], 1.0, (3,), float32)

Initial State


## Implementation

### DQN

### PPO

In [6]:
observation, info = env.reset(seed=42)
frames = []
for _ in range(10):
	action = env.action_space.sample()
	observation, reward, terminated, truncated, info = env.step(action)

	frames.append(observation)

	if terminated or truncated:
		observation, info = env.reset()
env.close()

show_animation(frames)